In [2]:
!pip install pandas

In [3]:
!pip install numpy

In [1]:
import pandas as pd
import numpy as np

In [2]:
import os
import sys

In [3]:
import tensorflow as tf

In [4]:
from keras import models
from keras.models import load_model

Using TensorFlow backend.


In [5]:
s=pd.read_csv('C:/Users/omomk/Downloads/train.csv')

In [6]:
s.info

<bound method DataFrame.info of                           qid  \
0        00002165364db923c7e6   
1        000032939017120e6e44   
2        0000412ca6e4628ce2cf   
3        000042bf85aa498cd78e   
4        0000455dfa3e01eae3af   
...                       ...   
1306117  ffffcc4e2331aaf1e41e   
1306118  ffffd431801e5a2f4861   
1306119  ffffd48fb36b63db010c   
1306120  ffffec519fa37cf60c78   
1306121  ffffed09fedb5088744a   

                                             question_text  target  
0        How did Quebec nationalists see their province...       0  
1        Do you have an adopted dog, how would you enco...       0  
2        Why does velocity affect time? Does velocity a...       0  
3        How did Otto von Guericke used the Magdeburg h...       0  
4        Can I convert montra helicon D to a mountain b...       0  
...                                                    ...     ...  
1306117  What other technical skills do you need as a c...       0  
1306118  Does MS in

In [7]:
embedding_index={}
g= open('C:/Users/omomk/Downloads/glove.6B.50d.txt',encoding='utf-8')
for line in g:
  values=line.split()
  word=values[0]
  coef=np.asarray(values[1:],dtype='float32')
  embedding_index[word]=coef
g.close

<function TextIOWrapper.close()>

In [8]:
!pip install sklearn

In [9]:
from sklearn.model_selection import train_test_split

In [10]:
x=s['question_text']
y=s['target']
x_train,x_test,y_train,y_test= train_test_split(x,y,test_size=0.2)

In [11]:
from sklearn.metrics import roc_auc_score, accuracy_score

In [12]:
from sklearn.utils import class_weight

#class_weight: Optional dictionary mapping class indices (integers) to a weight (float) value, used for weighting the loss function (during training only)

In [13]:
class_weight=class_weight.compute_class_weight('balanced',np.unique(y_train),y_train)

C:\Users\omomk\anaconda3\envs\tensorflow\lib\site-packages\sklearn\utils\validation.py:71: FutureWarning: Pass classes=[0 1], y=681820    0
382555    0
411625    1
657663    0
885760    0
         ..
749835    0
694341    0
511719    0
834230    0
390126    0
Name: target, Length: 1044897, dtype: int64 as keyword args. From version 0.25 passing these as positional arguments will result in an error
  FutureWarning)


In [14]:
print(class_weight)

[0.53300629 8.0743142 ]


In [15]:
class_weight_dict=dict(enumerate(class_weight))
class_weight_dict

{0: 0.5330062885638732, 1: 8.07431419519357}

In [16]:
!pip install nltk

In [17]:
from keras.preprocessing.text import Tokenizer

In [18]:
from nltk import word_tokenize

In [19]:
import nltk

In [20]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\omomk\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [21]:
sent_lens=[len(word_tokenize(x)) for x in x_train]
print(max(sent_lens))

412


In [22]:
print(np.percentile(sent_lens,99))

45.0


In [23]:
max_len=45

tk=Tokenizer(char_level=False,split=' ')
tk.fit_on_texts(x_train)
print(x_train.shape)

(1044897,)


In [24]:
seq_train=tk.texts_to_sequences(x_train)
seq_test=tk.texts_to_sequences(x_test)
print(len(seq_test),len(seq_test))
seq_train

261225 261225


[[16, 12, 38, 37, 52, 2734],
 [2, 12, 1, 889, 521, 13, 1535, 6, 962, 224, 1817, 13, 1810, 4492, 10, 5275],
 [11, 14, 67, 19, 37, 6, 1, 894, 253, 41, 199, 42, 8340, 27, 5805],
 [2, 3, 28, 558, 45, 1115, 8, 618, 376, 8760],
 [11, 37, 27, 1301, 24, 29, 21323, 2929],
 [2, 11, 14, 226, 28, 525, 32, 47, 7090, 6, 22, 14, 176, 170],
 [612, 9, 1, 41955, 286, 1143, 21, 1, 6615, 7, 418],
 [16,
  49,
  18,
  517,
  3296,
  502,
  21,
  204,
  18380,
  32,
  66,
  603,
  89,
  254,
  3,
  25,
  53,
  5,
  171],
 [9, 72, 31, 8, 867, 22, 18, 3854, 3145, 13, 18, 122, 103, 7, 5004],
 [287, 3348, 23, 7852, 30, 41, 151, 1, 287, 5, 33, 12731, 1129, 103, 498],
 [2, 31, 25, 254, 45, 1, 185, 2016, 1980, 6, 1, 92],
 [16, 12, 20740, 556, 4149],
 [16, 11, 52, 68, 430, 3203, 2314, 21324, 37, 46, 6709, 55, 386, 46, 55, 3187],
 [61, 15, 8, 91, 88281, 11587, 28435, 132, 4745, 2146, 3591],
 [20,
  8,
  24,
  29,
  2573,
  1262,
  3814,
  821,
  21,
  3010,
  8624,
  12273,
  1226,
  10,
  29890,
  8624,
  9,
  15,
 

 allows to vectorize a text corpus, by turning each text into either a sequence of integers (each integer being the index of a token in a dictionary) or into a vector where the coefficient for each token could be binary, based on word count, based on tf-idf.

By default, all punctuation is removed, turning the texts into space-separated sequences of words (words maybe include the ' character). These sequences are then split into lists of tokens. They will then be indexed or vectorized.

In [25]:
seq_train[0]

[16, 12, 38, 37, 52, 2734]

In [26]:
vocab_size=len(tk.word_index)

In [27]:
from keras.preprocessing import sequence

In [28]:
seq_train_pad=sequence.pad_sequences(seq_train,maxlen=max_len)
seq_test_pad=sequence.pad_sequences(seq_test,maxlen=max_len)

In [29]:
from keras.layers import LSTM, Activation, Dense, Input, Embedding, Dropout

In [30]:
embedding_matrix=np.zeros((vocab_size + 1, 50))

for word,i in tk.word_index.items():
    embed_vector=embedding_index.get(word)
    if embed_vector is not None:
        embedding_matrix[i]=embed_vector

In [31]:
from keras.models import Model

In [32]:
inputs=Input(name='text_input',shape=[max_len])
embed=Embedding(vocab_size + 1, 50,input_length=max_len,mask_zero=True,
                weights=[embedding_matrix],trainable=False)(inputs)
lstm1=LSTM(512)(embed)
drop1 = Dropout(0.2)(lstm1)
lstm2 = Dense(256,activation='relu')(drop1)
drop2 = Dropout(0.2)(lstm2)
lstm3 = Dense(128,activation='relu')(drop2)
drop3 = Dropout(0.2)(lstm3)
dense1 = Dense(25,activation='relu')(lstm3)
drop4 = Dropout(0.2)(dense1)
final_layer = Dense(1,activation='sigmoid')(drop4)

model = Model(inputs=inputs,outputs=final_layer)

In [33]:
model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
text_input (InputLayer)      (None, 45)                0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 45, 50)            9803300   
_________________________________________________________________
lstm_1 (LSTM)                (None, 512)               1153024   
_________________________________________________________________
dropout_1 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 256)               131328    
_________________________________________________________________
dropout_2 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 128)               3289

In [34]:
from keras.callbacks import ModelCheckpoint
from keras.callbacks import EarlyStopping

In [35]:
outputFolder = 'C:/Users/omomk/Downloads/spam_model_output'

In [36]:
filepath=os.path.join(outputFolder+"/weights-{epoch:02d}-{accuracy:.4f}.h5")
checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, 
                             save_best_only=True)

In [37]:
earlystop = EarlyStopping(monitor='val_loss', patience=3,
                          verbose=1)

path to save the model file. filepath can contain named formatting options, which will be filled the value of epoch and keys in logs (passed in on_epoch_end). For example: if filepath is weights.{epoch:02d}-{val_loss:.2f}.hdf5, then the model checkpoints will be saved with the epoch number and the validation loss in the filename

In [38]:
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

In [39]:
model.fit(seq_train_pad,y_train,
              epochs=5,
              class_weight={0: 0.5328573351855138, 1: 8.108651115146436},
              batch_size=10000,
              validation_data=(seq_test_pad,y_test),
              callbacks = [earlystop, checkpoint])

Train on 1044897 samples, validate on 261225 samples
Epoch 1/5
1044897/1044897 [==============================] - 3411s 3ms/step - loss: 0.4095 - accuracy: 0.7985 - val_loss: 0.3187 - val_accuracy: 0.8582

Epoch 00001: val_loss improved from inf to 0.31873, saving model to C:/Users/omomk/Downloads/spam_model_output/weights-01-0.7985.h5
Epoch 2/5
1044897/1044897 [==============================] - 5195s 5ms/step - loss: 0.3394 - accuracy: 0.8473 - val_loss: 0.3471 - val_accuracy: 0.8489

Epoch 00002: val_loss did not improve from 0.31873
Epoch 3/5
1044897/1044897 [==============================] - 3435s 3ms/step - loss: 0.3148 - accuracy: 0.8628 - val_loss: 0.2543 - val_accuracy: 0.8902

Epoch 00003: val_loss improved from 0.31873 to 0.25431, saving model to C:/Users/omomk/Downloads/spam_model_output/weights-03-0.8628.h5
Epoch 4/5
1044897/1044897 [==============================] - 5206s 5ms/step - loss: 0.3002 - accuracy: 0.8714 - val_loss: 0.3661 - val_accuracy: 0.8396

Epoch 00004: val

In [41]:
best_model = load_model(outputFolder+"/weights-05-0.8757.h5")

In [42]:
best_model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
text_input (InputLayer)      (None, 45)                0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 45, 50)            9803300   
_________________________________________________________________
lstm_1 (LSTM)                (None, 512)               1153024   
_________________________________________________________________
dropout_1 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 256)               131328    
_________________________________________________________________
dropout_2 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 128)               3289

In [43]:
from sklearn.metrics import roc_auc_score

In [45]:
p = best_model.predict(seq_test_pad)
roc_auc_score(y_test,p)



0.948941403124729

In [46]:
p

array([[0.7522913 ],
       [0.33860773],
       [0.17928815],
       ...,
       [0.97803247],
       [0.17597033],
       [0.00400825]], dtype=float32)

In [66]:
df=pd.DataFrame(p)

In [67]:
df.to_csv('pred1.csv',index=False)

In [68]:
model.save_weights('prj3_lstm_spam.h5')